In [ ]:
import re
import google.generativeai as genai
from youtube_transcript_api import YouTubeTranscriptApi
import os

# Configure Gemini API key

genai.configure(api_key="YOUR_API_KEY")

# Initialize the model
model = genai.GenerativeModel("gemini-1.5-flash")

def extract_video_id(url):
    """
    Extracts the video ID from a YouTube URL using regex.
    """
    match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
    return match.group(1) if match else None

def fetch_transcript(video_id):
    """
    Fetches the transcript for a given video ID using YouTube Transcript API.
    """
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        # Combine transcript pieces into a single string
        transcript = " ".join([item['text'] for item in transcript_list])
        return transcript
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

def summarize_text(text):
    """
    Summarizes the given text using the Gemini model.
    """
    prompt = f"Summarize the following YouTube transcript in detail (give as much details as possible):\n\n{text}\n\nSummary:"
    response = model.generate_content(prompt)
    return response.text

def summarize_youtube_video(url):
    """
    Automates the entire process of extracting video ID, fetching transcript,
    and generating summary for a given YouTube URL.
    """
    # Step 1: Extract video ID
    video_id = extract_video_id(url)
    if not video_id:
        return "Invalid YouTube URL"

    # Step 2: Fetch transcript
    transcript = fetch_transcript(video_id)
    if not transcript:
        return "Transcript not available"

    # Step 3: Summarize transcript
    summary = summarize_text(transcript)
    return summary

# Example usage
youtube_url = "Youtube_URL"
summary = summarize_youtube_video(youtube_url)
print("Summary:", summary)
